### This section is for functions I am going to need and all data processing down to a d_ci dataframe. That frame represents data for clustering. Changes are imminent, but this block sets everything up for analysis

Consider breaking out further

In [1]:
import os
import re
import math
import config
import kmeans
import extract
import constants
import matplotlib as mpl
import numpy as np
import pandas as pd
import transform
import plotly as py
import sys

#reload(sys)
py.tools.set_credentials_file(username='luci1408', api_key='dME8WYW9UKDs7cNWXPKm')

mpl.use('TkAgg')

files_location = config.data_source_file_location
files = os.listdir(files_location)

###############################################
# Functions which will eventually find a better home
#
pd.Series.filter_by = kmeans.filter_by
pd.DataFrame.filter_by = kmeans.filter_by


# Now we must clean the data for modeling. Think about recording transformations and informationa bout them

Cluster hotels within market by:
    -Distance to other hotels (how many hotels are within an x radius of this one)
    -Operating Revenue 'ANSWER_ann_op_rev'
    -Size of meeting space or meetings related 'ANSWER_sq_ft_meeting'
    -Number of Rooms 'ANSWER_num_rooms'
    -RevPAR 'ANSWER_ann_revenue/ANSWER_num_rooms*365'
    -STR Chain Scale? Diamonds? Stars?

In [2]:
# Register transform functions here, create them in the transform.py
column_operations = {'ANSWER_num_rooms':transform.answer_num_rooms,
                     'ANSWER_ann_op_rev':transform.answer_ann_op_rev,
                     'ANSWER_ann_revenue':transform.answer_ann_revenue,
                     'ANSWER_num_employees_pos':transform.answer_num_employees_pos,
                     'ANSWER_cur_base_pay':transform.answer_cur_base_pay,
                     'create_RevPAR':transform.create_RevPAR,
                     'create_Executive':transform.create_Executive}


d_ci = extract.extract_all_lazy()
d_ci_v2  = transform.clean_hotel_model(d_ci, column_operations)



# The top X number of markets by the number of properties
number_of_markets = 15
number_of_clusters = 3
markets = kmeans.top_markets_by_property(number_of_markets, d_ci_v2)
positions = d_ci_v2['POSITION'].unique().tolist()

kmeans.py:108: FutureWarning:

sort(columns=....) is deprecated, use sort_values(by=.....)



In [3]:
math.ceil(11.0/5.0)

3.0

In [4]:
# This section calculates the clusters for the hotels

total_frame = pd.DataFrame()
distance_frame = pd.DataFrame()

for market in markets:
    for job in positions:
        temp_frame = d_ci_v2[d_ci_v2['CITYMARKET'] == market]
        temp_frame = temp_frame[temp_frame['POSITION'] == job]
        if temp_frame.shape[0] > 1:
            temp_frame = temp_frame.fillna(np.nan)
            temp_frame['clusters'] = ''

            print market + " " + job + " "# + str(temp_frame['CREATED_cur_base_pay_hrs'])

            #this needs to be in a function like so:
            #cluster_positions(temp_frame)
            pcode = temp_frame['PROPERTY_CODE'].values
#             f1 = np.linalg.norm(temp_frame['ANSWER_num_rooms'].astype(float).values)
#             f2 = np.linalg.norm(temp_frame['CREATED_revpar'].astype(float).values)
#             f3 = np.linalg.norm(temp_frame['ANSWER_num_employees_pos'].astype(int).values)

            temp_frame['rank_sum'] = .5*f1 + .5*f3
            temp_frame = temp_frame.sort_values(by=['rank_sum'])
            
            temp_matrix = np.matrix((f1,f2,f3))
            

            #print temp_frame['ANSWER_num_rooms'].astype(float).values
            #test = kmeans.run_equal_size_kmeans(temp_matrix, num_obs, number_of_clusters, num_obs_per_cluster)
#             temp_frame_km = kmeans.kmeans_missing(temp_matrix, number_of_clusters, max_iter=500)
#             means = kmeans.ChooseInitialMeans(temp_matrix)
#             distmat = kmeans.SetDistMat(temp_matrix, means)
#             clusters = kmeans.InitialAssignment(distmat, num_obs_per_cluster, num_of_clusters)

            min_obs = 5
            num_obs = temp_frame.shape[0]
            num_obs_per_cluster = math.ceil(float(num_obs)/float(number_of_clusters))
            num_obs_mod = num_obs_per_cluster % min_obs
            
            print "number of observations: %s" % num_obs
            print "number of obs per cluster: %s" % num_obs_per_cluster

            if num_obs < 5:
                temp_frame['clusters'] = 'No'
            elif num_obs >= 5 and num_obs < 10:
                temp_frame['clusters'] = '0'
            elif num_obs > 10:
                for g, df in temp_frame.groupby(np.arange(len(temp_frame)) // num_obs_per_cluster):
                    temp_frame.ix[df.index,'clusters'] = g
#             elif num_obs == 10:
#                 for g, df in temp_frame.groupby(np.arange(len(temp_frame)) // (min_obs)):
#                     temp_frame.ix[df.index,'clusters'] = g
#             elif num_obs == 11:
#                 for g, df in temp_frame.groupby(np.arange(len(temp_frame)) // (min_obs+1)):
#                     temp_frame.ix[df.index,'clusters'] = g
#             elif num_obs >= 12 and num_obs <= 14:
#                 for g, df in temp_frame.groupby(np.arange(len(temp_frame)) // (min_obs+2)):
#                     temp_frame.ix[df.index,'clusters'] = g
#             elif num_obs == 15:
#                 for g, df in temp_frame.groupby(np.arange(len(temp_frame)) // (min_obs)):
#                     temp_frame.ix[df.index,'clusters'] = g
#             elif num_obs == 16:
#                 for g, df in temp_frame.groupby(np.arange(len(temp_frame)) // (num_obs_per_cluster+2)):
#                     temp_frame.ix[df.index,'clusters'] = g                       
#             elif num_obs > 17:
#                 for g, df in temp_frame.groupby(np.arange(len(temp_frame)) // (num_obs_per_cluster)):
#                     temp_frame.ix[df.index,'clusters'] = g
                        
            total_frame = total_frame.append(temp_frame)
            
            if num_obs > 11:
                pass
                #print temp_frame[['PROPERTY_NAME','clusters']]
    else:
        total_frame = total_frame


San Diego Accounting Manager 
[[ 242.78591392  190.59237617    1.41421356]]
number of observations: 2
number of obs per cluster: 1.0
San Diego Administrative Assistant/Department Secretary 
[[ 2204.87822793   542.03813109    29.69848481]]
number of observations: 11
number of obs per cluster: 4.0
San Diego Administrative Assistant Executive/Secretary 
[[ 844.25766209  486.14966592    7.61577311]]
number of observations: 8
number of obs per cluster: 3.0
San Diego Accounting Clerk, Entry 
[[ 525.83837821  340.90128182    2.64575131]]
number of observations: 4
number of obs per cluster: 2.0
San Diego Accounts Payable Clerk 
[[ 1194.58779502   412.945534       3.        ]]
number of observations: 6
number of obs per cluster: 2.0
San Diego Banquet Server/Waiter/Waitress 
[[ 2177.32863849   645.42967989   210.23558215]]
number of observations: 14
number of obs per cluster: 5.0
San Diego Banquet Set-Up/House Person 
[[ 2170.20367708   633.05860607    48.43552415]]
number of observations: 13
nu

In [5]:
f2

745.86899479089482

# This section will be used to calculate summary statistics, diagnostics, and model outputs

In [6]:
jobs_frame = total_frame
jobs_frame['CREATED_cur_base_pay_hrs'] = jobs_frame['CREATED_cur_base_pay_hrs'].astype(float)
j_frame = jobs_frame.groupby(['CITYMARKET','POSITION','clusters'])['CREATED_cur_base_pay_hrs'].describe(include='all').reset_index()
j_frame = pd.pivot_table(j_frame, index = ['CITYMARKET','POSITION','clusters'], columns = ['level_3'], aggfunc=np.sum).reset_index()
j_frame.columns = ['CITYMARKET','POSITION','clusters','25','50','75','count','max','mean','min','std']
j_frame = j_frame.sort_values(by=['CITYMARKET','POSITION', 'clusters','count'])

hotels_frame = total_frame[['PROPERTY_CODE','PROPERTY_NAME','CITYMARKET','STATE','ZIP','POSITION','clusters']]

In [7]:
############################
# This is to be turned into a function which creates the output for reporting. 
# an xlsx which will contain summary and detailed info for a 'top market' run. 
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('ww_top_' + str(number_of_markets) + '_markets_output.xlsx', engine='xlsxwriter')

for market in markets:
    
    jobs_sheet = j_frame[j_frame['CITYMARKET'] == market]
    jobs_sheet_name = market + "_" + "wages"
    jobs_sheet.to_excel(writer, sheet_name = jobs_sheet_name, encoding='utf8')

    hotels_sheet = hotels_frame[hotels_frame['CITYMARKET'] == market]
    hotels_sheet_name = market + "_" + "hotels"
    hotels_sheet.to_excel(writer, sheet_name = hotels_sheet_name, encoding='utf8')
    

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [8]:
x = data_ci.columns.tolist()
x.append('cluster')
x

NameError: name 'data_ci' is not defined

# This section will be for visual analysis of the clustering. 

In [ ]:
hotel_cluster_frame['text'] = hotel_cluster_frame['Clusters'] + ' ' + hotel_cluster_frame['PROPERTY_NAME'] + 'Num of Rooms: ' + hotel_cluster_frame['ANSWER_num_rooms']

scl = [ [0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
    [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"] ]

data = [ dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = hotel_cluster_frame['BINGLONGITUDE'],
        lat = hotel_cluster_frame['BINGLATITUDE'],
        text = hotel_cluster_frame['text'],
        mode = 'markers',
        marker = dict(
            size = 8,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale = scl,
            cmin = 0,
            color = hotel_cluster_frame['Clusters'],
            cmax = hotel_cluster_frame['Clusters'].max(),
            colorbar=dict(
                title="Clusters"
            )
        ))]

layout = dict(
        title = 'Hotels by Cluster',
        colorbar = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5
        ),
    )

fig = dict( data=data, layout=layout )
py.plotly.iplot( fig, validate=False, filename='d3-hotel-clusters')

In [ ]:
#new_frame
new_frame.groupby('Clusters')['Clusters','ANSWER_ann_op_rev'].head(100)#['ANSWER_ann_op_rev'].head()
new_frame

In [ ]:
markets

In [ ]:
#d_ci[d_ci['ANSWER_ann_revenue'].isnull()].shape
#d_ci.head()

long_lat = ['PROPERTY_CODE','BINGLATITUDE','BINGLONGITUDE']
d_ci_dist = d_ci[long_lat]
d_ci_dist.set_index(d_ci_dist['PROPERTY_CODE'])

d_ci_dist_v1 = pd.DataFrame(distance_matrix(d_ci_dist.values, d_ci_dist.values), index=d_ci_dist['PROPERTY_CODE'], columns=d_ci_dist['PROPERTY_CODE'])

###############################################
# create a distance matrix


In [ ]:
d_ci_dist_v2 = d_ci_dist_v1.reset_index()

The recorded answers to the questions "What is the location's Diamond Rating" include the answer, "Yes" and "No". 823 observations between the two of them or roughly 23% of all of the answers (3,541). 1,359 observations are essentially N/A

In [ ]:
gi_unique_lists.keys()
d = data_gi[data_gi['QUESTION'] == "What is the location's Diamond Rating?"].groupby(['ANSWER']).count().reset_index()
d.sort_values(['PROPERTY_CODE'], ascending = False)

#general info diamond star
gi_ds = data_gi[data_gi['QUESTION'].str.contains("Diamond Rating?")]
                #data_gi['QUESTION']=="What is the location's Star Rating?"]

gi_ds = pd.DataFrame(gi_ds)

d['PROPERTY_CODE'].sum()
#gi_ds[gi_ds['ANSWER'].str.contains("Star", na = False)]

Looking at the 2016 dataset for General Info tabs and focusing on the questions, "What is the location's Diamond Rating" and "What is the location's Star Rating". I find the number of observations between the two to differ a bit. i.e. not all properties had any response recorded to both questions. This could be a known with your team.

The Star rating question includes "Diamonds", "No", and "Yes" as answers

3,754 answers total 2,442 N/A



In [ ]:
d = data_gi[data_gi['QUESTION'] == "What is the location's Star Rating?"].groupby(['ANSWER']).count().reset_index()
#d[d['ANSWER'].str.contains('Diamond')].sort(['PROPERTY_CODE'], ascending = False)

data_stars = data_gi[data_gi['QUESTION'] == "What is the location's Star Rating?"]
data_stars_v1 = data_stars[(data_stars['ANSWER'].str.contains('Diamond', na = False))]#.groupby(['ANSWER']).count().reset_index()
#data_stars_v2  = data_stars_v1[(data_stars_v1['ANSWER'].str.contains('Not Applicable', na = False))]
#data_stars_out = data_stars[data_stars['ANSWER'].str.contains('Diamonds', na = False)]
#data_stars
#d[['ANSWER','PROPERTY_CODE']]
#data_stars_out.to_csv("LocationStarRating_Diamonds_2016.csv", index = False)

data_stars_v1.groupby(['ANSWER']).count().reset_index()

In [ ]:
data_stars_v1.to_csv("LocationStarRating_Diamonds_2016.csv",index=False)

In [ ]:
data_ci_cols = data_ci.columns.tolist()
data_pi_cols = data_pi.columns.tolist()
data_gi_cols = data_gi.columns.tolist()

citymarket_frame = data_pi.groupby(['CITYMARKET']).count()
cmf1 = citymarket_frame.reset_index().sort(['PROPERTY_CODE'], ascending = False).head(40)
topmarkets = cmf1['CITYMARKET'].tolist()

In [ ]:
np.nanmean.func_defaults